In [73]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd

from tool import *
from scipy.signal import resample
from scipy import signal
from scipy.signal import butter, lfilter, freqz, freqs
from scipy.ndimage import label

import heartpy as hp

import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

saved_path = "./Result"

# Create folders
if not os.path.exists("Result/CVA_analysis/Bland_Altman"): os.makedirs("Result/CVA_analysis/Bland_Altman")
if not os.path.exists("Result/CVA_analysis/Dataframe"): os.makedirs("Result/CVA_analysis/Dataframe")
if not os.path.exists("Result/CVA_analysis/Extraction_all"): os.makedirs("Result/CVA_analysis/Extraction_all")
if not os.path.exists("Result/CVA_analysis/Figures"): os.makedirs("Result/CVA_analysis/Figures")
if not os.path.exists("Result/CVA_analysis/Results_all"): os.makedirs("Result/CVA_analysis/Results_all")
if not os.path.exists("Result/EDA_mat"): os.makedirs("Result/EDA_mat")
if not os.path.exists("Result/EDA_mat_analysis"): os.makedirs("Result/EDA_mat_analysis")

%matplotlib widget
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Create list with the path of each participant to analyze
paths = []

for p in range(7, 31):
    for e in range(1, 3):
        fc_path = "./Data/Participant 200"+np.str(p)+"/Flex comp/"+np.str(e)+".txt"
        em_path = "./Data/Participant 200"+np.str(p)+"/EmotiBit/Parsed/_00"+np.str(e)
        name = "P"+np.str(p)+"-"+np.str(e)
        
        paths.append([em_path, fc_path, name])

In [39]:
###############################################################################
# Extract PPG, EDA from Emotibit and ECG, EDA from Flexcomp data
###############################################################################
for path in paths:
    
    s = path[2]
    
    sig = ["PI", "PG", "PR", "GX", "GY", "GZ", "AX", "AY", "AZ", "EA", "ER", "EL", "B%"]

    if int(path[2][1:-2]) <= 12:
        if int(path[2][1:-2]) == 12:
            sig = ["PI", "PG", "PR", "GX", "GY", "GZ", "AX", "AY", "AZ", "EA", "ER", "EL"]
        else:
            sig = ["PI", "PG", "PR", "GX", "GY", "GZ", "AX", "AY", "AZ", "EA", "ER", "EL", "B%"]
            
        em_df = load_merge_signals_7_12(path[0], sig, ppg_filter=True, G_filter=True)
        fc_df = load_flexcomp(path[1])
        em_pos, fc_pos, em_df, fc_df, error = sync_pos_without_button(em_df, fc_df)
    elif int(path[2][1:-2]) <= 18:
        sig = ["PI", "PG", "PR", "GX", "GY", "GZ", "AX", "AY", "AZ", "EA", "ER", "EL"]
        em_df = load_merge_signals(path[0], sig, ppg_filter=True, G_filter=True)
        fc_df = load_flexcomp(path[1])
        em_pos, fc_pos, em_df, fc_df, error = sync_pos_without_button(em_df, fc_df)
    else:
        em_df = load_merge_signals(path[0], sig, ppg_filter=True, G_filter=True)
        fc_df = load_flexcomp(path[1])
        em_pos, fc_pos, em_df, fc_df, error = sync_pos(em_df, fc_df)

    sig = fc_df["EKG-Pro/Flex - 1A"].values
    norm_sig = (sig - sig.mean()) / sig.std()

    fc_df["NORM_ECG"] = norm_sig

    ###############################################################################
    # Merge all PPG signals
    ###############################################################################
    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    sig = (em_df["PR"].values+em_df["PI"].values+em_df["PG"].values)/3
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    # Normalize signal with mean 0 and unit variance
    norm_sig = (sig - sig.mean()) / sig.std()

    em_df["NORM_PPG"] = norm_sig

    final_df = fc_df[["Time"]].copy()
    final_df["ECG_FC"] = fc_df[["EKG-Pro/Flex - 1A"]]
    final_df["EDA_FC"] = fc_df[["SC-Pro/Flex - 1E"]]
    final_df["NORM_ECG"] = fc_df[["NORM_ECG"]]
    try:
        final_df = pd.merge_asof(final_df, em_df, on="Time", direction="nearest", tolerance=0.0015)
    except:
        print(s, ":Fail in Merge!")
        continue
        
    ###############################################################################
    # Add EDA signal
    ###############################################################################
    
    ind, value = adjust_EDA(final_df[["EA", "ER", "EL"]][final_df["EA"].notna()], EL_only=False)
    final_df.at[ind, "EA"] = value
        
    df_eda = stress_detector(final_df, resample="100L", normalization="no_norm", corr_sig="smooth")
    final_df = merge_nearest(final_df, df_eda, "Time", df_eda.columns[1:])

    df = final_df
    
    final_df.to_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

    ###############################################################################
    # ECG
    ###############################################################################
    ecg = df[["Time","NORM_ECG"]][df["NORM_ECG"].notna()].copy()
    ecg["Time_ts"] = pd.to_datetime(ecg['Time'], unit='s')
    ecg = ecg.resample("10L", on="Time_ts").mean()
    ecg = ecg.interpolate(limit_direction='forward', axis=0)
    
    fs = 100

    filtered = hp.filter_signal(ecg["NORM_ECG"].values, cutoff = 0.05, sample_rate = fs, filtertype='notch')

    filtered = resample(filtered, len(filtered) * 2)

    wd_fc, m_fc = hp.process(hp.scale_data(filtered), fs*2, windowsize=0.75)

    np.save(saved_path + "/CVA_analysis/Extraction_all/{}_FC".format(s), [wd_fc, m_fc])

    HR_FC = get_HR_series(wd_fc, fs=200)

    plt.figure(1)
    plot = hp.plotter(wd_fc, m_fc, show=False)
    plt.savefig(saved_path + "/CVA_analysis/Figures/{}_ECG.png".format(s))
    plt.close(1)

    ###############################################################################
    # PPG
    ###############################################################################
    ppg = df[["Time","NORM_PPG"]][df["NORM_PPG"].notna()].copy()
    ppg["Time_ts"] = pd.to_datetime(ppg['Time'], unit='s')
    ppg = ppg.resample("10L", on="Time_ts").mean()
    ppg = ppg.interpolate(alimit_direction='forward', axis=0)

    if int(path[2][1:-2]) <= 18:
        filtered = hp.filter_signal(ppg["NORM_PPG"].values[2500:-2000], cutoff = [0.7, 3.5], sample_rate = fs, order=3, filtertype='bandpass')
    else:
        filtered = hp.filter_signal(ppg["NORM_PPG"].values[400:-400], cutoff = [0.7, 3.5], sample_rate = fs, order=3, filtertype='bandpass')

    if int(path[2][1:-2]) == 24:
        filtered[filtered>0.8] = 0.8
        filtered[filtered<-0.8] = -0.8
        
    filtered = resample(filtered, len(filtered) * 2)
        
    wd_em, m_em = hp.process(filtered, fs*2, windowsize=1)

    np.save(saved_path + "/CVA_analysis/Extraction_all/{}_EM".format(s), [wd_em, m_em])

    
    plt.figure(1)
    hp.plotter(wd_em, m_em, show=False)
    plt.savefig(saved_path + "/CVA_analysis/Figures/{}_PPG.png".format(s))
    plt.close(1)

    HR_EM = get_HR_series(wd_em, fs=200)

    np.save(saved_path + "/CVA_analysis/Extraction_all/{}_EM_FC_HR".format(s), [HR_EM, HR_FC])

    p1 = plt.figure(1)
    plt.plot(np.arange(0, HR_FC[2:].shape[0]*2, 2), HR_FC[2:])
    plt.plot(np.arange(0, HR_EM[:].shape[0]*2, 2), HR_EM[:], color="orange", linestyle='dashed')
    plt.ylim([0,180])
    plt.xlabel("Time (s)")
    plt.ylabel("HR (bpm)")
    plt.title("HR example for a 10-minute session")
    plt.legend(["Reference device (ECG)", "Emotibit device (PPG)"])

    plt.savefig(saved_path + "/CVA_analysis/Figures/{}_HR.svg".format(s))
    plt.close('all')
    
    print(s, m_fc["bpm"], m_em["bpm"], m_fc["bpm"]-m_em["bpm"])

PI
PG
PR
GX
GY
GZ
AX
AY
AZ
EA
ER
EL
B%


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

P30-2 41.993757684668495 71.93223226539207 -29.93847458072358


In [41]:
#######################################################
# Calculate SQI
#######################################################
list_SQI_approved = []
good_seg = []

for part in range(7,31):
    for exp in range(1,3):
        s = "P{}-{}".format(part, exp)

        if s == "P18-2":
            continue
        
        try:
            wd, m = np.load(saved_path + "/CVA_analysis/Extraction_all/{}_EM.npy".format(s), allow_pickle=True)
        except:
            continue

        data_all = wd["hr"]

        peaks_all = list(wd["peaklist"])
        rejected_peaks = list(wd["removed_beats"])

        for p in rejected_peaks:
            peaks_all.remove(p)

        peaks_all = np.array(peaks_all)

        R2_all = []
        SQI_all = []
        
        fs = 200

        for i in range(0, data_all.shape[0], 10*fs):
            data = np.array(data_all[i:i+10*fs])

            peaks = np.copy(peaks_all[np.where((peaks_all<i+10*fs)&(peaks_all>i))[0]])
            peaks -= i

            R2, SQI = sqi_calculator(data, peaks, fs, 0.86, 10, verbose=0)

            R2_all.append(R2)
            SQI_all.append(SQI)

        SQI_all = np.array(SQI_all)
        R2_all = np.array(R2_all)

        print("{} : SQI>0.86 : {}/{} -> {:.0f}%".format(s, np.sum(SQI_all==1), len(SQI_all), np.sum(SQI_all==1)/len(SQI_all)*100))
        
        if np.sum(SQI_all==1)/len(SQI_all)*100 > 50:
            good_seg.append(np.sum(SQI_all==1)/len(SQI_all)*100)
            list_SQI_approved.append(s)
            
np.save(saved_path + "/CVA_analysis/List_SQI_approved.npy", list_SQI_approved)

P7-1 : SQI>0.86 : 54/63 -> 86%
P7-2 : SQI>0.86 : 58/62 -> 94%
P8-1 : SQI>0.86 : 47/73 -> 64%
P8-2 : SQI>0.86 : 5/19 -> 26%
P9-1 : SQI>0.86 : 46/66 -> 70%
P9-2 : SQI>0.86 : 60/63 -> 95%
P10-1 : SQI>0.86 : 62/62 -> 100%
P10-2 : SQI>0.86 : 70/80 -> 88%
P11-1 : SQI>0.86 : 61/62 -> 98%
P11-2 : SQI>0.86 : 59/61 -> 97%
P12-1 : SQI>0.86 : 60/62 -> 97%
P12-2 : SQI>0.86 : 61/61 -> 100%
P13-1 : SQI>0.86 : 58/62 -> 94%
P13-2 : SQI>0.86 : 54/60 -> 90%
P14-1 : SQI>0.86 : 58/58 -> 100%
P14-2 : SQI>0.86 : 57/58 -> 98%
P15-1 : SQI>0.86 : 17/18 -> 94%
P16-1 : SQI>0.86 : 53/58 -> 91%
P16-2 : SQI>0.86 : 55/58 -> 95%
P17-1 : SQI>0.86 : 51/58 -> 88%
P17-2 : SQI>0.86 : 52/58 -> 90%
P18-1 : SQI>0.86 : 55/58 -> 95%
P19-1 : SQI>0.86 : 5/60 -> 8%
P19-2 : SQI>0.86 : 54/60 -> 90%
P20-1 : SQI>0.86 : 60/61 -> 98%
P20-2 : SQI>0.86 : 60/60 -> 100%
P21-1 : SQI>0.86 : 59/60 -> 98%
P21-2 : SQI>0.86 : 57/60 -> 95%
P22-1 : SQI>0.86 : 61/62 -> 98%
P22-2 : SQI>0.86 : 61/61 -> 100%
P23-1 : SQI>0.86 : 60/60 -> 100%
P23-2 : SQI

In [72]:
print(list_SQI_approved)

['P7-1' 'P7-2' 'P8-1' 'P9-1' 'P9-2' 'P10-1' 'P10-2' 'P11-1' 'P11-2'
 'P12-1' 'P12-2' 'P13-1' 'P13-2' 'P14-1' 'P14-2' 'P15-1' 'P16-1' 'P16-2'
 'P17-1' 'P17-2' 'P18-1' 'P19-2' 'P20-1' 'P20-2' 'P21-1' 'P21-2' 'P22-1'
 'P22-2' 'P23-1' 'P23-2' 'P24-1' 'P24-2' 'P25-1' 'P25-2' 'P26-1' 'P26-2'
 'P27-1' 'P27-2' 'P28-1' 'P28-2' 'P29-1' 'P29-2']


In [43]:
list_SQI_approved = ['P7-1',
                     'P7-2',
                     'P8-1',
                     'P9-1',
                     'P9-2',
                     'P10-1',
                     'P10-2',
                     'P11-1',
                     'P11-2',
                     'P12-1',
                     'P12-2',
                     'P13-1',
                     'P13-2',
                     'P14-1',
                     'P14-2',
                     'P15-1',
                     'P16-1',
                     'P16-2',
                     'P17-1',
                     'P17-2',
                     'P18-1',
                     'P19-2',
                     'P20-1',
                     'P20-2',
                     'P21-1',
                     'P21-2',
                     'P22-1',
                     'P22-2',
                     'P23-1',
                     'P23-2',
                     'P24-1', # Added
                     'P24-2',
                     'P25-1',
                     'P25-2',
                     'P26-1',
                     'P26-2',
                     'P27-1',
                     'P27-2',
                     'P28-1',
                     'P28-2',
                     'P29-1',
                     'P29-2']

np.save(saved_path + "/CVA_analysis/List_SQI_approved.npy", list_SQI_approved)

In [44]:
print("Number of approved participants : ", len(list_SQI_approved))
print("The average percentage of good segments : ", np.mean(good_seg), "%")

Number of approved participants :  42
The average percentage of good segments :  91.03785016188627 %


In [46]:
# Plot of PPG and ECG signals
ecg = (wd_fc["hr"]-np.mean(wd_fc["hr"]))/np.std(wd_fc["hr"])
ppg = (wd_em["hr"]-np.mean(wd_em["hr"]))/np.std(wd_em["hr"])

plt.figure()
plt.plot(ecg[804:])
plt.plot(ppg[158:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [55]:
###########################################################
# Extraction of the HR, RR, SDRR and RMSSD from ECG and PPG
###########################################################
HR_FC_all_exp = []
SD_RR_FC_all_exp = []
RMSSD_FC_exp = []

HR_EM_all_exp = []
SD_RR_EM_all_exp = []
RMSSD_EM_exp = []

list_SQI_approved = np.load(saved_path + "/CVA_analysis/List_SQI_approved.npy")

for s in list_SQI_approved:
    try:
        wd_fc, m_fc = np.load(saved_path + "/CVA_analysis/Extraction_all/{}_FC.npy".format(s), allow_pickle=True)
    except:
        continue

    HR_FC_all = 60000/(np.mean(wd_fc["RR_list_cor"]))
    HR_FC = get_HR_series(wd_fc)
    HR_FC = interp_nan(HR_FC)
    SD_RR_FC_all = np.std(wd_fc["RR_list_cor"])/1000
    SD_RR_FC_all = m_fc["sdnn"]
    RMSSD_FC = get_RMSSD(wd_fc)
    RMSSD_FC = m_fc["rmssd"]

    HR_FC_all_exp.append(HR_FC_all)
    SD_RR_FC_all_exp.append(SD_RR_FC_all)
    RMSSD_FC_exp.append(RMSSD_FC)

    wd_em, m_em = np.load(saved_path + "/CVA_analysis/Extraction_all/{}_EM.npy".format(s), allow_pickle=True)

    HR_EM_all = 60000/(np.mean(wd_em["RR_list_cor"]))
    HR_EM = get_HR_series(wd_em)
    HR_EM = interp_nan(HR_EM)
    SD_RR_EM_all = np.std(wd_em["RR_list_cor"])/1000
    SD_RR_EM_all = m_em["sdnn"]
    RMSSD_EM = get_RMSSD(wd_em)
    RMSSD_EM = m_em["rmssd"]

    HR_EM_all_exp.append(HR_EM_all)
    SD_RR_EM_all_exp.append(SD_RR_EM_all)
    RMSSD_EM_exp.append(RMSSD_EM)

    if HR_FC.shape[0] - HR_EM.shape[0] < 0:
        diff = np.abs(HR_FC.shape[0] - HR_EM.shape[0])
        HR_EM = HR_EM[:-diff]
    elif HR_FC.shape[0] - HR_EM.shape[0] > 0:
        diff = np.abs(HR_FC.shape[0] - HR_EM.shape[0])
        HR_FC = HR_FC[:-diff]


    MAE = np.mean(np.abs(HR_FC-HR_EM))
    SdAE = np.std(np.abs(HR_FC-HR_EM))
    ARE = ((np.abs(HR_FC-HR_EM)/HR_FC)*100).mean()

    np.save(saved_path + "/CVA_analysis/Results_all/{}_FC.npy".format(s), [HR_FC_all, HR_FC, SD_RR_FC_all, RMSSD_FC, MAE, SdAE, ARE])
    np.save(saved_path + "/CVA_analysis/Results_all/{}_EM.npy".format(s), [HR_EM_all, HR_EM, SD_RR_EM_all, RMSSD_EM, MAE, SdAE, ARE])

    print("{} -> MAE:{:.2f}, SdAE:{:.2f}, ARE:{:.2f}, HR_mean:{:.2f};{:.2f}, SD_RR:{:.2f};{:.2f}, RMSSD:{:.2f};{:.2f}".format(s, MAE, SdAE, ARE, HR_EM_all, HR_FC_all, SD_RR_EM_all, SD_RR_FC_all, RMSSD_EM*1000, RMSSD_FC*1000))

np.save(saved_path + "/CVA_analysis/Bland_Altman/HR_mean.npy", [HR_EM_all_exp, HR_FC_all_exp])
np.save(saved_path + "/CVA_analysis/Bland_Altman/SD_RR.npy", [SD_RR_EM_all_exp, SD_RR_FC_all_exp])
np.save(saved_path + "/CVA_analysis/Bland_Altman/RMSSD.npy", [RMSSD_EM_exp, RMSSD_FC_exp])

P7-1 -> MAE:3.87, SdAE:3.61, ARE:8.54, HR_mean:44.03;44.09, SD_RR:144.51;141.70, RMSSD:110954.75;109263.09
P7-2 -> MAE:4.10, SdAE:3.97, ARE:9.16, HR_mean:43.51;43.55, SD_RR:143.61;140.77, RMSSD:120240.85;113055.15
P8-1 -> MAE:5.52, SdAE:4.55, ARE:8.96, HR_mean:62.31;61.42, SD_RR:119.30;90.56, RMSSD:164948.50;74333.81
P9-1 -> MAE:3.31, SdAE:3.50, ARE:4.31, HR_mean:74.81;75.83, SD_RR:51.65;40.48, RMSSD:44270.14;28205.09
P9-2 -> MAE:3.14, SdAE:2.87, ARE:4.24, HR_mean:73.24;73.56, SD_RR:50.76;39.14, RMSSD:50242.16;31889.81
P10-1 -> MAE:6.72, SdAE:5.46, ARE:7.91, HR_mean:83.61;83.42, SD_RR:63.16;64.26, RMSSD:35708.37;31692.26
P10-2 -> MAE:6.00, SdAE:5.65, ARE:7.19, HR_mean:80.89;81.81, SD_RR:67.65;62.74, RMSSD:53543.75;31007.23
P11-1 -> MAE:6.79, SdAE:5.36, ARE:11.02, HR_mean:60.84;60.67, SD_RR:104.46;98.31, RMSSD:77230.22;71473.47
P11-2 -> MAE:5.19, SdAE:4.51, ARE:8.75, HR_mean:59.04;59.61, SD_RR:104.88;103.60, RMSSD:84619.31;65223.03
P12-1 -> MAE:5.92, SdAE:4.67, ARE:8.58, HR_mean:69.16;6

# Bland-Altman CVA Analysis

In [56]:
import pingouin

list_SQI_approved = np.load(saved_path + "/CVA_analysis/List_SQI_approved.npy")

HR_EM_ba = []
HR_FC_ba = []

for s in list_SQI_approved:
    try:
        [HR_FC_all, HR_FC, SD_RR_FC_all, RMSSD_FC, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_FC.npy".format(s), allow_pickle=True)
        [HR_EM_all, HR_EM, SD_RR_EM_all, RMSSD_EM, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_EM.npy".format(s), allow_pickle=True)
    except:
        continue

    HR_EM_ba.append(HR_EM_all)
    HR_FC_ba.append(HR_FC_all)

HR_EM_ba = np.array(HR_EM_ba).reshape([-1])
HR_FC_ba = np.array(HR_FC_ba).reshape([-1])

lies_in = np.sum(((HR_EM_ba-HR_FC_ba)<5) & ((HR_EM_ba-HR_FC_ba)>-5))/np.size(HR_EM_ba)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
pingouin.plot_blandaltman(HR_EM_ba, HR_FC_ba, confidence=None, ax=ax)
plt.axhline(5, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-5, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("HR (bpm)")
plt.ylabel("Difference HR (EM - FC)")
plt.xlabel("Mean HR (EM,FC)")

SD = np.std((HR_EM_ba-HR_FC_ba))
BAr = SD*1.96/(np.mean((HR_EM_ba+HR_FC_ba)/2))
print(BAr)

plt.text(0.75, 0.9, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)

plt.savefig(saved_path + "/CVA_analysis/Figures/BA_HR.svg", format="svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.023348250199029388


In [57]:
SD_EM_ba = []
SD_FC_ba = []

for s in list_SQI_approved:
    try:
        [HR_FC_all, HR_FC, SD_RR_FC_all, RMSSD_FC, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_FC.npy".format(s), allow_pickle=True)
        [HR_EM_all, HR_EM, SD_RR_EM_all, RMSSD_EM, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_EM.npy".format(s), allow_pickle=True)
    except:
        continue

    SD_EM_ba.append(SD_RR_EM_all)
    SD_FC_ba.append(SD_RR_FC_all)

SD_EM_ba = np.array(SD_EM_ba).reshape([-1])
SD_FC_ba = np.array(SD_FC_ba).reshape([-1])

lies_in = np.sum(((SD_EM_ba-SD_FC_ba)<60) & ((SD_EM_ba-SD_FC_ba)>-60))/np.size(SD_EM_ba)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax = pingouin.plot_blandaltman(SD_EM_ba, SD_FC_ba, confidence=0.1, ax=ax)
plt.axhline(60, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-60, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("SD RR/PP - interval (msec)")
plt.ylabel("Difference SD RR/PP (EM - FC)")
plt.xlabel("Mean SD RR/PP (EM,FC)")

SD = np.std((SD_EM_ba-SD_FC_ba))
BAr = SD*1.96/(np.mean((SD_EM_ba+SD_FC_ba)/2))
print(BAr)

plt.text(0.75, 0.9, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.savefig(saved_path + "/CVA_analysis/Figures/BA_SD.svg", format="svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.18186141881520673


In [59]:

RMSSD_EM_ba = []
RMSSD_FC_ba = []

for s in list_SQI_approved:
    try:
        [HR_FC_all, HR_FC, SD_RR_FC_all, RMSSD_FC, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_FC.npy".format(s), allow_pickle=True)
        [HR_EM_all, HR_EM, SD_RR_EM_all, RMSSD_EM, MAE, SdAE, ARE] = np.load(saved_path + "/CVA_analysis/Results_all/{}_EM.npy".format(s), allow_pickle=True)
    except:
        continue

    RMSSD_EM_ba.append(RMSSD_EM)
    RMSSD_FC_ba.append(RMSSD_FC)

RMSSD_EM_ba = np.array(RMSSD_EM_ba).reshape([-1])
RMSSD_FC_ba = np.array(RMSSD_FC_ba).reshape([-1])

lies_in = np.sum(((RMSSD_EM_ba-RMSSD_FC_ba)<70) & ((RMSSD_EM_ba-RMSSD_FC_ba)>-70))/np.size(RMSSD_EM_ba)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
pingouin.plot_blandaltman(RMSSD_EM_ba, RMSSD_FC_ba, confidence=0.1, ax=ax)
plt.axhline(70, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-70, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("RMSSD (msec)")
plt.ylabel("Difference RMSSD (EM - FC)")
plt.xlabel("Mean RMSSD (EM,FC)")

SD = np.std((RMSSD_EM_ba-RMSSD_FC_ba))
BAr = SD*1.96/(np.mean((RMSSD_EM_ba+RMSSD_FC_ba)/2))
print(BAr)

plt.text(0.75, 0.9, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.savefig(saved_path + "/CVA_analysis/Figures/BA_RMSSD.svg", format="svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.633082149938286


# EDA Analysis

## Visual inspection of EDA data from Emotibit vs Flexcomp

In [62]:
for part in range(7,31):
    for exp in range(1,3):
        s = "P{}-{}".format(part, exp)
        try:
            df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))
        except:
            continue
        
        EDA_FC = df[["EDA_FC_SMO"]][df["EDA_FC_SMO"].notna()].values[1:-250]
        EDA_EM = df[["EDA_EM_SMO"]][df["EDA_EM_SMO"].notna()].values[1:-250]

        EDA_FC[0,0] += 0.0000001
        
        EDA_FC_norm_dt = normalize_detrend(EDA_FC)
        EDA_EM_norm_dt = normalize_detrend(EDA_EM)
        
        plt.figure()
        plt.title(s)
        plt.plot(EDA_FC_norm_dt)
        plt.plot(EDA_EM_norm_dt)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
list_EDA_approved = ["P7-1",
                     "P7-2",
                     "P10-1",
                     "P10-2",
                     "P11-1",
                     "P11-2",
                     "P13-1",
                     "P13-2",
                     "P14-1",
                     "P14-2",
                     "P17-1",
                     "P17-2",
                     "P18-1",
                     "P19-2",
                     "P20-1",
                     "P20-2",
                     "P21-1",
                     "P21-2",
                     "P22-1",
                     "P22-2",
                     "P23-1",
                     "P23-2",
                     "P24-1",
                     "P24-2",
                     "P25-1",
                     "P25-2",
                     "P26-1",
                     "P26-2",
                     "P28-1",
                     "P28-2",
                     "P29-1",
                     "P29-2",
                     "P30-1",
                     "P30-2"]

In [65]:
eda_corrcoef = []

for s in list_EDA_approved:
    df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))
    
    EDA_FC = df[["EDA_FC_SMO"]][df["EDA_FC_SMO"].notna()].values[100:-250]
    EDA_EM = df[["EDA_EM_SMO"]][df["EDA_EM_SMO"].notna()].values[100:-250]
    
    EDA_FC_norm_dt = normalize_detrend(EDA_FC)
    EDA_EM_norm_dt = normalize_detrend(EDA_EM)
    
    c = cross_corr_lags(EDA_EM_norm_dt, EDA_FC_norm_dt, lags=100) # We selected a larger lags because data are not well synchronized
    
    eda_corrcoef.append(c)
    
print("The average value of the correlation is {}".format(np.mean(eda_corrcoef)))
    
plt.figure(figsize=(12,4))
plt.hist(eda_corrcoef, bins=[0, 0.2, 0.4, 0.6, 0.8, 1], weights=np.ones(len(eda_corrcoef))/len(eda_corrcoef))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.ylim([0,1])
plt.xlabel("Cross correlation coefficient")
plt.ylabel("Percentage of participants")
plt.savefig(saved_path + "/CVA_analysis/Figures/Hist_Corr.svg", format="svg")

67 0.9399800964167473
61 0.922524342068039
38 0.8534261011980706
42 0.8284050812429699
37 0.9885643100815023
31 0.9811274327147749
100 0.9244503144178703
48 0.8063129729226468
28 0.9643359983832788
3 0.9809361483487499
0 0.656543394007799
28 0.8517863606203321
38 0.9266533567288486
15 0.7015298144501937
-3 0.9943505592285837
-1 0.9933892304423353
36 0.9851616458883331
7 0.9752746521082362
8 0.42127835934399727
7 0.947833372164087
-4 0.9385464472068509
-11 0.9644805681321795
5 0.9569346588370555
10 0.9004090103805399
-9 0.9594754450908732
-18 0.9602568397678171
-6 0.573322633317157
-7 0.9725099719213568
4 0.8985310163730442
7 0.39522746656439306
-100 0.9307621742971968
-63 0.8534016108518864
10 0.8942146681985109
19 0.8748358144759062
La valeur moyenne de la corrélation est de 0.8740227020056518


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
for s in list_EDA_approved:

    df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

    print(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

    EDA_FC = df[["EDA_FC_NORM"]][df["EDA_FC_NORM"].notna()].values[100:-250]
    EDA_EM = df[["EDA_EM_NORM"]][df["EDA_EM_NORM"].notna()].values[100:-250]
    
    EDA_FC = df[["EDA_FC_SMO"]][df["EDA_FC_SMO"].notna()].values[100:-250]
    EDA_EM = df[["EDA_EM_SMO"]][df["EDA_EM_SMO"].notna()].values[100:-250]
    
    plt.figure()
    plt.title(s)
    plt.plot(EDA_FC)
    plt.plot(EDA_EM)

./Result/CVA_analysis/Dataframe/P7-1.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Result/CVA_analysis/Dataframe/P7-2.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Result/CVA_analysis/Dataframe/P10-1.pkl


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Result/CVA_analysis/Dataframe/P30-1.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Result/CVA_analysis/Dataframe/P30-2.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Example of best and worst EDA collection

In [67]:
s = "P20-1"
    
df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

print(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

EDA_FC = df[["EDA_FC_NORM","Time"]][df["EDA_FC_NORM"].notna()].values[100:-250]
EDA_EM = df[["EDA_EM_NORM","Time"]][df["EDA_EM_NORM"].notna()].values[100:-250]

EDA_FC = df[["EDA_FC_SMO","Time"]][df["EDA_FC_SMO"].notna()].values[100:-250]
EDA_EM = df[["EDA_EM_SMO","Time"]][df["EDA_EM_SMO"].notna()].values[100:-250]

plt.figure()
plt.title("Highest cross-correlation (0.99)")
plt.plot(EDA_FC[:,1], EDA_FC[:,0])
plt.plot(EDA_EM[:,1], EDA_EM[:,0])
plt.legend(["Reference device", "Emotibit device"])
plt.ylim([0,4])
plt.xlabel("Time (s)")
plt.ylabel("Skin conductance (µS)")
plt.savefig(saved_path + "/CVA_analysis/Figures/Highest_Corr.svg", format="svg")

s = "P28-2"
    
df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

print(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))

EDA_FC = df[["EDA_FC_NORM","Time"]][df["EDA_FC_NORM"].notna()].values[100:-250]
EDA_EM = df[["EDA_EM_NORM","Time"]][df["EDA_EM_NORM"].notna()].values[100:-250]

EDA_FC = df[["EDA_FC_SMO","Time"]][df["EDA_FC_SMO"].notna()].values[100:-250]
EDA_EM = df[["EDA_EM_SMO","Time"]][df["EDA_EM_SMO"].notna()].values[100:-250]

plt.figure()
plt.title("Lowest cross-correlation (0.42)")
plt.plot(EDA_FC[:,1], EDA_FC[:,0])
plt.plot(EDA_EM[:,1], EDA_EM[:,0])
plt.legend(["Reference device", "Emotibit device"])
plt.ylim([0,8])
plt.xlabel("Time (s)")
plt.ylabel("Skin conductance (µS)")
plt.savefig(saved_path + "/CVA_analysis/Figures/Lowest_Corr.svg", format="svg")


./Result/CVA_analysis/Dataframe/P20-1.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

./Result/CVA_analysis/Dataframe/P28-2.pkl


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# The remaining analysis of the EDA is done on Matlab with Ledalab

In [74]:
################################################
# Conversion of EDA data into matlab files (.mat)
################################################
from scipy.io import savemat

for p in list_EDA_approved:
    try:
        x = np.load(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(p), allow_pickle=True)
    except:
        continue
        
    fileinfo = {"version": 3.4900,
                "date": [2021, 1, 26, 17, 12, 26.1370],
                "log": []}

    EDA = x[["EDA_FC_SMO"]][x["EDA_FC_SMO"].notna()].values[100:-250]
    Time = x[["Time"]][x["EDA_FC_SMO"].notna()].values[100:-250]

    event = {"time" : [],
             "nid" : [],
             "name" : [],
             "userdata" : []}

    data = {"conductance" : EDA,
            "time" : Time,
            "timeoff" : 0,
            "event" : event}

    mdic = {"data": data}

    savemat(saved_path + "/EDA_mat/DATA_{}_EDA_FC.mat".format(p), mdic)
    
    EDA = x[["EDA_EM_SMO"]][x["EDA_EM_SMO"].notna()].values[100:-250]
    Time = x[["Time"]][x["EDA_EM_SMO"].notna()].values[100:-250]

    event = {"time" : [],
             "nid" : [],
             "name" : [],
             "userdata" : []}

    data = {"conductance" : EDA,
            "time" : Time,
            "timeoff" : 0,
            "event" : event}

    mdic = {"data": data}

    savemat(saved_path + "/EDA_mat/DATA_{}_EDA_EM.mat".format(p), mdic)

## Steps to extract the number of SCRs and the amplitude of each SCR
1. Copy the EDA_mat folder to the main Ledalab folder.
2. Run the matlab command: Ledalab('Path to EDA_mat folder\', 'open', 'mat', 'analyze', 'CDA')
3. Rename the folder "EDA_mat_analysis" and copy the "EDA_mat_analysis" folder to the python environment. Analysis data is inside the same files given in input.

Matlab instruction:
Ledalab('Path to EDA_mat folder\', 'open', 'mat', 'analyze', 'CDA')

# Use Python to plot Bland-Altman graphes

In [75]:
import scipy

SCL_EM = []
SCL_FC = []

for s in list_EDA_approved:
    try:
        df = pd.read_pickle(saved_path + "/CVA_analysis/Dataframe/{}.pkl".format(s))
    except:
        continue

    EDA_FC = df[["EDA_FC_SMO"]][df["EDA_FC_SMO"].notna()].values[100:-250]
    EDA_EM = df[["EDA_EM_SMO"]][df["EDA_EM_SMO"].notna()].values[100:-250]

    SCL_EM.append(EDA_EM.mean())
    SCL_FC.append(EDA_FC.mean())

logSCL_EM = np.array(SCL_EM).reshape([-1])
logSCL_FC = np.array(SCL_FC).reshape([-1])

lies_in = np.sum(((logSCL_EM-logSCL_FC)<1.6) & ((logSCL_EM-logSCL_FC)>-1.6))/np.size(logSCL_EM)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
pingouin.plot_blandaltman(logSCL_EM, logSCL_FC, confidence=0.1, ax=ax)
plt.axhline(1.6, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-1.6, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("Mean SCL")
plt.ylabel("Difference SCL (EM - FC)")
plt.xlabel("Mean SCL (EM,FC)")

SD = np.std((logSCL_EM-logSCL_FC))
BAr = SD*1.96/(np.mean((logSCL_EM+logSCL_FC)/2))
print(BAr)
plt.text(0.75, 0.9, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.savefig(saved_path + "/CVA_analysis/Figures/BA_SCL.svg", format="svg")

# Number of SCRs
nSCR_EM = []
nSCR_FC = []

S_ampl_EM = []
S_ampl_FC = []

for s in list_EDA_approved:
    try:
        ana_em = scipy.io.loadmat(saved_path + "/EDA_mat_analysis/DATA_{}_EDA_EM.mat".format(s))
        ana_fc = scipy.io.loadmat(saved_path + "/EDA_mat_analysis/DATA_{}_EDA_FC.mat".format(s))
    except:
        continue
    
    nSCR_EM.append(ana_em["analysis"]["amp"][0,0].shape[1]/10)
    nSCR_FC.append(ana_fc["analysis"]["amp"][0,0].shape[1]/10)

    S_ampl_EM.append(ana_em["analysis"]["amp"][0,0].sum()/10)
    S_ampl_FC.append(ana_fc["analysis"]["amp"][0,0].sum()/10)
    

lognSCR_EM = np.array(nSCR_EM).reshape([-1])
lognSCR_FC = np.array(nSCR_FC).reshape([-1])

lies_in = np.sum(((lognSCR_EM-lognSCR_FC)<2.5) & ((lognSCR_EM-lognSCR_FC)>-2.5))/np.size(lognSCR_EM)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
pingouin.plot_blandaltman(lognSCR_EM, lognSCR_FC, confidence=0.1, ax=ax)
plt.axhline(2.5, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-2.5, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("Mean noSCRs")
plt.ylabel("Difference noSCRs (EM - FC)")
plt.xlabel("Mean noSCRs (EM,FC)")

SD = np.std((lognSCR_EM-lognSCR_FC))
BAr = SD*1.96/(np.mean((lognSCR_EM+lognSCR_FC)/2))
print(BAr)
plt.text(0.75, 0.9, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.savefig(saved_path + "/CVA_analysis/Figures/BA_noSCRs.svg", format="svg")

logS_ampl_EM = np.array(S_ampl_EM).reshape([-1])
logS_ampl_FC = np.array(S_ampl_FC).reshape([-1])

lies_in = np.sum(((logS_ampl_EM-logS_ampl_FC)<0.6) & ((logS_ampl_EM-logS_ampl_FC)>-0.6))/np.size(logS_ampl_EM)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
pingouin.plot_blandaltman(logS_ampl_EM, logS_ampl_FC, confidence=0.1, ax=ax)
plt.axhline(0.6, xmin=0, xmax=1, c="g", linestyle="-.")
plt.axhline(-0.6, xmin=0, xmax=1, c="g", linestyle="-.")
plt.text(0.5, 0.1, '{} % of the data lies inside of the boundaries.'.format(int(lies_in*100)), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.title("Mean S-AMPL")
plt.ylabel("Difference S-AMPL (EM - FC)")
plt.xlabel("Mean S-AMPL (EM,FC)")

SD = np.std((logS_ampl_EM-logS_ampl_FC))
BAr = SD*1.96/(np.mean((logS_ampl_EM+logS_ampl_FC)/2))
print(BAr)
plt.text(0.75, 0.85, 'BAr : {:.2f}'.format(BAr), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=10)
plt.savefig(saved_path + "/CVA_analysis/Figures/BA_S-AMPL.svg", format="svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0066681676078535


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.5184027091629658


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.8558382115571397
